In [1]:
from collections import deque

## Part 1

In [2]:
test = [
"px{a<2006:qkq,m>2090:A,rfg}",
"pv{a>1716:R,A}",
"lnx{m>1548:A,A}",
"rfg{s<537:gd,x>2440:R,A}",
"qs{s>3448:A,lnx}",
"qkq{x<1416:A,crn}",
"crn{x>2662:A,R}",
"in{s<1351:px,qqz}",
"qqz{s>2770:qs,m<1801:hdj,R}",
"gd{a>3333:R,R}",
"hdj{m>838:A,pv}",
"",
"{x=787,m=2655,a=1222,s=2876}",
"{x=1679,m=44,a=2067,s=496}",
"{x=2036,m=264,a=79,s=2244}",
"{x=2461,m=1339,a=466,s=291}",
"{x=2127,m=1623,a=2188,s=1013}",
]

In [3]:
def parse_workflow(text):
    
    workflow = []

    rule_name, rules = text[:-1].split("{")
    for rule in rules.split(","):

        if ":" in rule:
            rule = rule.split(":")

            if "<" in rule[0]:
                category, value = rule[0].split("<")
                sign = "<"
            else:
                category, value = rule[0].split(">")
                sign = ">"
            
            workflow.append([category, sign, int(value), rule[1]])
        
        else:
            workflow.append(rule)
        
    return {rule_name: workflow}

In [4]:
def parse_text(text):
    workflow =  {}
    parts = []

    is_workflow = True
    for row in text:

        if not row:
            is_workflow=False
            continue

        if is_workflow:
            workflow.update(parse_workflow(row))
        else:
            row = {i.split("=")[0]: int(i.split("=")[1]) for i in row[1:-1].split(",")}
            parts.append(row)
    
    return workflow, parts

In [5]:
def update_workflow(workflow, items):
    next_workflow = None
    
    
    for part, operator, value, wf in workflow[:-1]:
        
        if part in items:

            if operator == "<" and items[part] < value:
                next_workflow = wf
            elif operator == ">" and items[part] > value:
                next_workflow = wf

        if next_workflow:
            return next_workflow
                            
    return workflow[-1]

In [6]:
def follow_workflow(workflow, items):

    q = [workflow['in']]

    while q:
        current = q.pop()
        next_workflow = update_workflow(current, items)

        if next_workflow == "A":
            return sum(items.values())
        elif next_workflow == "R":
            return 0
        else:
            q.append(workflow[next_workflow])

In [7]:
workflow, parts = parse_text(test)

In [8]:
results = []

for part in parts:
    result = follow_workflow(workflow, part)
    results.append(result)

In [9]:
assert sum(results) == 19114

In [10]:
text = open("../advent_of_code_input/2023/day_19.txt", "r").readlines()
text = [i.split("\n")[0] for i in text]

In [11]:
workflow, parts = parse_text(text)

In [12]:
results = []

for part in parts:
    result = follow_workflow(workflow, part)
    results.append(result)

In [13]:
sum(results)

480738

## Part 2

In [14]:
workflow, parts = parse_text(test)

In [15]:
def split_interval(interval, operator, value):
        
    if operator == "<":
        
        if interval[0] < value and interval[1] > value:
            
            return [interval[0], value-1], [value, interval[1]]
        
        elif interval[1] < value:
                
                return [], interval
            
        else:
            
            return interval, []
        
    else:

        if interval[0] < value and interval[1] > value:
            
            return [value+1, interval[1]], [interval[0], value]
        
        elif interval[1] < value:
                
                return [], interval
        
        else:
            
            return interval, []

        
def calculate_combinations(categories):
    
    results = 0
    
    for category in categories:
        result = 1
        for i,j in category.values():
            result *= (j-i)+1

        results += result

    return results

In [16]:
def get_accepted_range(workflow):

    results = []
    categories = {"x": [1,4000], "a": [1,4000], "s": [1,4000], "m": [1,4000]}

    q = deque([(workflow["in"], categories)])

    while q:

        for _ in range(len(q)):

            rules, categories = q.popleft()
            
            for var, op, val, rule_name in rules[:-1]:

                category = categories[var]
                interval_1, interval_2 = split_interval(category, op, val)

                if interval_1 and interval_2:

                    new_categories = categories.copy()
                    new_categories[var] = interval_1

                    if rule_name == "A":
                        results.append(new_categories)
                    elif rule_name == "R":
                        pass
                    else:
                        q.append([workflow[rule_name], new_categories])

                    categories[var] = interval_2

            if rules[-1] == "A":
                results.append(categories)
            elif rules[-1] == "R":
                continue
            else:
                q.append((workflow[rules[-1]], categories))

    return results

In [17]:

workflow, parts = parse_text(test)
results = get_accepted_range(workflow)
results = calculate_combinations(results)

In [18]:
assert results == 167409079868000

In [19]:
text = open("../advent_of_code_input/2023/day_19.txt", "r").readlines()
text = [i.split("\n")[0] for i in text]

In [20]:
workflow, parts = parse_text(text)

In [21]:

workflow, parts = parse_text(text)
results = get_accepted_range(workflow)
results = calculate_combinations(results)

In [22]:
results

131550418841958